In [23]:
# Setup
import os, re, ast, time
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
%pip install -q sentence-transformers

# Define file paths
comments_files = [
    r"C:\Users\taneg\OneDrive\Desktop\datathon\data\comments1.csv"
]
videos_path = r"C:\Users\taneg\OneDrive\Desktop\datathon\data\videos.csv"


# Artifacts folder (for saving/loading model)
ARTIFACT_DIR = "commentsense_hybrid_artifacts"
MODEL_VEC_PATH = f"{ARTIFACT_DIR}/tfidf_vec.joblib"
MODEL_CLF_PATH = f"{ARTIFACT_DIR}/logreg_clf.joblib"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

print("✅ Setup done.")


Note: you may need to restart the kernel to use updated packages.
✅ Setup done.


## 🔮 Note on Translation Demo

All **data cleaning and preprocessing steps** are already fully defined and implemented in our main pipeline files.  
The code shown here is **not part of the core pipeline** — it is a **future planning demonstration**.  

The purpose is to show that we can already **translate non-English comments into English** at scale.  
For this hackathon, we translated **1,000 comments** as a proof-of-concept.  
In future iterations, this can be integrated directly into the main pipeline to ensure **multilingual coverage across all markets**.


In [24]:
# Install and setup NLTK VADER with SSL workaround
%pip install -q nltk

import nltk
import ssl
import urllib.request

# Fix SSL certificate issues for NLTK downloads
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Download VADER lexicon
try:
    nltk.download('vader_lexicon', quiet=True)
    print("VADER lexicon downloaded successfully")
except Exception as e:
    print(f"Download failed: {e}")
    # Alternative: try downloading to specific directory
    try:
        nltk_data_dir = os.path.expanduser('~/nltk_data')
        os.makedirs(nltk_data_dir, exist_ok=True)
        nltk.download('vader_lexicon', download_dir=nltk_data_dir, quiet=True)
        print("VADER lexicon downloaded to user directory")
    except Exception as e2:
        print(f"Alternative download also failed: {e2}")
        print("You may need to download manually or use a different sentiment analyzer")


Note: you may need to restart the kernel to use updated packages.
VADER lexicon downloaded successfully


In [25]:
import ssl
import certifi
import nltk

# Use system certificates
ssl_context = ssl.create_default_context(cafile=certifi.where())
ssl._create_default_https_context = lambda: ssl_context

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\taneg\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [26]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [27]:
# Initialize VADER Sentiment Analyzer
from nltk.sentiment import SentimentIntensityAnalyzer

try:
    vader = SentimentIntensityAnalyzer()
    print("VADER Sentiment Analyzer initialized successfully!")
    
    # Test it with a sample text
    test_text = "This is amazing! I love it!"
    scores = vader.polarity_scores(test_text)
    print(f"Test scores for '{test_text}': {scores}")
    
except Exception as e:
    print(f"Failed to initialize VADER: {e}")
    print("You may need to manually download the vader_lexicon or use an alternative")


VADER Sentiment Analyzer initialized successfully!
Test scores for 'This is amazing! I love it!': {'neg': 0.0, 'neu': 0.259, 'pos': 0.741, 'compound': 0.8619}


In [28]:
# Keep relevant columns: text from comments + metadata from videos
def stage1_prepare(df_comments, df_videos):
    merged = df_comments[['videoId','textOriginal']].merge(
        df_videos[['videoId','title','description','tags','topicCategories']], 
        on='videoId', how='left'
    )
    return merged

In [29]:
# Cleaning text (memory-safe: single pass per cell)
URL_RE        = re.compile(r'https?://\S+|www\.\S+', re.I)
HTML_TAG_RE   = re.compile(r'<.*?>', re.S)
MENTION_RE    = re.compile(r'@\w+')
HASHTAG_WORDS = re.compile(r'#(\w+)')
PUNCT_RE      = re.compile(r'[^\w\s]', re.U)
MULTISPACE_RE = re.compile(r'\s+')

def clean_one(s: str) -> str:
    if not isinstance(s, str):
        s = "" if pd.isna(s) else str(s)
    s = s.lower()
    s = URL_RE.sub(" ", s)
    s = HTML_TAG_RE.sub(" ", s)
    s = MENTION_RE.sub(" ", s)
    s = HASHTAG_WORDS.sub(r" \1 ", s)  # '#spf50' -> ' spf50 '
    s = PUNCT_RE.sub(" ", s)           # removes ascii punctuation
    s = MULTISPACE_RE.sub(" ", s).strip()
    return s

def tags_to_string(x):
    if pd.isna(x): return ""
    if isinstance(x, list): return ' '.join(map(str, x))
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list): return ' '.join(map(str, parsed))
        except: pass
    return str(x)

def parse_topics(x):
    if isinstance(x, list): 
        return [str(u).split('/')[-1] for u in x]
    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            if isinstance(v, list): return [str(u).split('/')[-1] for u in v]
        except: return []
    return []

def to_lower_list(x):
    if isinstance(x, list):
        return [str(t).lower() for t in x]
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return [str(t).lower() for t in parsed]
        except:
            return []
    return []

def stage2_clean(df):
    """
    Memory-safe cleaning:
    - avoid chained .str.replace (creates temporaries)
    - do one .map() with a pure-Python cleaner per column
    """
    df = df.copy()

    # Turn tags (list/JSON) into flat string first (temporary column)
    df['tags_str'] = df['tags'].map(tags_to_string)

    # Apply one-pass cleaner per column (no chain)
    df['textOriginal_clean'] = df['textOriginal'].map(clean_one)
    df['title_clean']        = df['title'].map(clean_one)
    df['description_clean']  = df['description'].map(clean_one)
    df['tags_clean']         = df['tags_str'].map(clean_one)

    # Topics → normalized lists + lowercase helper
    df['topicCategories_clean']    = df['topicCategories'].map(parse_topics)
    df['topicCategories_clean_lc'] = df['topicCategories_clean'].map(to_lower_list)

    # Free temporary to lower pressure
    del df['tags_str']
    return df


In [30]:
RELATED_TOPICS = {
    'entertainment','fashion','health','knowledge','business','technology',
    'lifestyle_(sociology)','physical_attractiveness','society',
    'performing_arts','pop_music','television_program','tourism'
}


In [31]:
def filter_related_topics(df):
    return df[
        df['topicCategories_clean_lc'].apply(
            lambda topics: any(t in RELATED_TOPICS for t in topics)
        )
    ].copy()

In [32]:
# Step 5: English detection (Strategy B — fast)
import re

def build_en_dic():
    EN_COMMON = {"the","a","an","i","you","u","he","she","we","they","it","me","my","mine","your","ur","our","ours","their","theirs",
                 "him","her","them","this","that","these","those",
                 "and","or","but","so","if","when","then","because","also","than","as",
                 "with","without","while","though",
                 "in","on","at","by","with","for","to","from","of","about","over","under","after","before",
                 "is","are","was","were","be","been","am","do","did","does","have","has","had","can","could","should","would","will",
                 "go","goes","went","gone","make","made","say","said","see","saw","seen","get","got","come","came","want","needs","need",
                 "use","used","try","tried","buy","bought","like","love","hate",
                 "good","bad","great","nice","beautiful","pretty","amazing","awesome","cool","cute","sweet","clean","soft","perfect","best",
                 "better","worse","really","very","super","ultra","mega","just","only","ever","always","again",
                 "not","no","yes","ok","okay","yeah","yep","yup","pls","please","plz",
                 "lol","lmao","rofl","haha","hehe","omg","wow","yay","thx","thanks","thank","sorry","congrats","wtf",
                 "video","post","pic","photo","review","tutorial","tips","hack","trend","color","colour","shade","price","stock","link",
                 "shop","buy","sale","deal","code","free","gift","new"}
    BEAUTY_EN = {"serum","moisturizer","cleanser","toner","essence","spf","sunscreen","retinol", "makeup",
                 "foundation","concealer","powder","lipstick","mascara","eyeliner","blush",
                 "perfume","fragrance","cologne","shampoo","conditioner","hair","scalp"}
    return EN_COMMON | BEAUTY_EN

EN_DIC = build_en_dic()
EN_PATTERN = r'\b(' + '|'.join([re.escape(w).replace(r"\ ", r"\s+") for w in sorted(EN_DIC)]) + r')\b'

def english_flag_series(s, strict_hits=2, loose_hits=4):
    s = s.fillna('').astype(str)
    ascii_mask = s.str.match(r'^[\x00-\x7F]+$', na=False)
    hits = s.str.count(EN_PATTERN, flags=re.IGNORECASE)
    return (ascii_mask & (hits>=strict_hits)) | ((~ascii_mask) & (hits>=loose_hits))

def split_english(df2):
    for c in ['textOriginal_clean','title_clean','description_clean','tags_clean']:
        df2[f"is_english__{c}"] = english_flag_series(df2[c])
    df2["is_english"] = df2[[f"is_english__{c}" for c in ['textOriginal_clean','title_clean','description_clean','tags_clean']]].any(axis=1)
    return df2[df2["is_english"]].copy(), df2[~df2["is_english"]].copy()


In [33]:
# Step 6: Stage 3a - Keyword categorization
CATEGORY_DICT = {
    "skincare": [
        "serum","moisturizer","cleanser","toner","essence","spf","sunscreen","retinol",
        "mask","exfoliate","acne","blemish","pore","cream","lotion","gel","facial","skin","skincare",
        "hydrating","whitening","brightening","anti-aging","antiwrinkle","wrinkle",
        "niacinamide","vitamin c","sensitive skin","hydration"
    ],
    "makeup": [
        "makeup","foundation","concealer","powder","lipstick","lip gloss","mascara","eyeliner","brow","blush",
        "bronzer","highlighter","palette","primer","setting spray","shade","contour","bb","cc","tint",
        "liner","lip balm","liquid lipstick","matte","glitter","lashes","dewy","smokey eye"
    ],
    "fragrance": [
        "perfume","fragrance","cologne","parfum","toilette","eau","edp","edt",
        "vanilla","floral","musk","scent","aroma","oud","citrus","woody","fresh"
    ],
    "hair": [
        "haircare","shampoo","conditioner","hair","hair mask","hair oil","dry shampoo",
        "frizz","keratin","scalp","dye","color","hairstyle","split ends","straightening","curling",
        "styling gel","mousse","leave-in","heat protectant","volume","shine","alopecia","haircut","hairstyles","haircare"
    ],
    "beauty_general": [
        "beauty","cosmetic","cosmetics","glam","beauty routine","beauty tips",
        "makeover","tutorial","skincare routine","self-care","glow","radiance",
        "fashion","runway","trend","outfit","lookbook","aesthetic",
        "hijab","hijab style","modest fashion"
    ]
}

CATEGORY_DICT = {k:[w.lower() for w in v] for k,v in CATEGORY_DICT.items()}

def build_cat_regex(words):
    pattern = r'\b(?:' + "|".join([re.escape(w).replace(r"\ ", r"\s+") for w in words]) + r')\b'
    return re.compile(pattern, re.IGNORECASE)

CATEGORY_REGEX = {cat: build_cat_regex(words) for cat, words in CATEGORY_DICT.items()}

def keyword_categorize(df_en):
    kw_text = (df_en["title_clean"].fillna("") + " " +
               df_en["description_clean"].fillna("") + " " +
               df_en["tags_clean"].fillna(""))
    # add na=False so missing values don’t propagate warnings
    masks = {cat: kw_text.str.contains(rx, na=False) for cat, rx in CATEGORY_REGEX.items()}
    df_en["category_kw"] = "other"
    for cat in ["skincare","makeup","fragrance","hair","beauty_general"]:
        df_en.loc[masks[cat] & (df_en["category_kw"]=="other"), "category_kw"] = cat
    return df_en


In [34]:
# Train TF-IDF + Logistic Regression
def train_tfidf_lr(train_df, sample_n=200_000, random_state=42):
    df = train_df[train_df["category_kw"]!="other"].copy()
    if len(df) > sample_n:
        df = df.sample(sample_n, random_state=random_state)

    X = (df["title_clean"].fillna("") + " " +
         df["description_clean"].fillna("") + " " +
         df["tags_clean"].fillna(""))
    y = df["category_kw"]

    Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

    vec = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2))
    Xtr_v = vec.fit_transform(Xtr)
    Xva_v = vec.transform(Xva)

    # liblinear defaults to one-vs-rest; drop multi_class to avoid FutureWarning
    clf = LogisticRegression(max_iter=250, solver="liblinear")
    clf.fit(Xtr_v, ytr)

    print("Validation:\n", classification_report(yva, clf.predict(Xva_v)))

    joblib.dump(vec, MODEL_VEC_PATH)
    joblib.dump(clf, MODEL_CLF_PATH)
    return vec, clf


In [35]:
# Step 8: ML inference + hybrid override
def infer_categories(df_en, vec, clf):
    X_all = (df_en["title_clean"].fillna("") + " " + df_en["description_clean"].fillna("") + " " + df_en["tags_clean"].fillna(""))
    X_all_v = vec.transform(X_all)
    df_en["category_ml"] = clf.predict(X_all_v)
    df_en["category_final"] = df_en["category_kw"]
    mask_other = df_en["category_final"].eq("other")
    df_en.loc[mask_other, "category_final"] = df_en.loc[mask_other, "category_ml"]
    return df_en




In [36]:
# Accumulate across files
all_df_en = pd.DataFrame()
all_df_non_en = pd.DataFrame()

df_videos = pd.read_csv(videos_path)  # load once

for i, path in enumerate(comments_files, start=1):
    t0 = time.time()
    print(f"\n=== Processing file {i}/{len(comments_files)} ===\n{path}")
    df_comments = pd.read_csv(path)

    # Stage 1 → Stage 2
    df1 = stage1_prepare(df_comments, df_videos)
    df2 = stage2_clean(df1)

    # 🔎 Apply topic filter here
    df2 = filter_related_topics(df2)
    print("After topic filter shape:", df2.shape)
    
    # English / non-English split
    df_en, df_non_en = split_english(df2)

    # Keyword categories
    df_en = keyword_categorize(df_en)

    # Train on first file, load for the rest
    if i == 1:
        vec, clf = train_tfidf_lr(df_en)
    else:
        vec, clf = joblib.load(MODEL_VEC_PATH), joblib.load(MODEL_CLF_PATH)

    # ML inference + hybrid override
    df_en = infer_categories(df_en, vec, clf)

    # Append to accumulators
    all_df_en = pd.concat([all_df_en, df_en], ignore_index=True)
    all_df_non_en = pd.concat([all_df_non_en, df_non_en], ignore_index=True)

    # Quick per-file summary
    n_other_kw = (df_en["category_kw"] == "other").sum()
    rescued = ((df_en["category_kw"] == "other") & (df_en["category_ml"] != "other")).sum()
    print(f"File {i}: EN rows={len(df_en):,} | NON-EN rows={len(df_non_en):,} | OTHER by KW={n_other_kw:,} | Rescued by ML={rescued:,}")
    print(f"Top categories (keyword):\n{df_en['category_kw'].value_counts().head(5)}")
    print(f"⏱ File runtime: {time.time()-t0:.1f}s")

print("\n✅ All files processed.")
print("English combined shape:", all_df_en.shape)
print("Non-English combined shape:", all_df_non_en.shape)



=== Processing file 1/1 ===
C:\Users\taneg\OneDrive\Desktop\datathon\data\comments1.csv
After topic filter shape: (988663, 12)
Validation:
                 precision    recall  f1-score   support

beauty_general       0.99      0.94      0.96       912
     fragrance       0.99      0.95      0.97       288
          hair       0.99      1.00      1.00      6514
        makeup       0.99      1.00      0.99     12431
      skincare       0.99      0.99      0.99      9855

      accuracy                           0.99     30000
     macro avg       0.99      0.97      0.98     30000
  weighted avg       0.99      0.99      0.99     30000

File 1: EN rows=910,737 | NON-EN rows=77,926 | OTHER by KW=103,990 | Rescued by ML=103,990
Top categories (keyword):
category_kw
makeup            333756
skincare          264866
hair              175560
other             103990
beauty_general     24772
Name: count, dtype: int64
⏱ File runtime: 1097.5s

✅ All files processed.
English combined shape: 

In [37]:
from collections import Counter
topic_counts = Counter(t for row in df2['topicCategories_clean_lc'] for t in row)
print("Top topicCategories kept:", pd.Series(topic_counts).sort_values(ascending=False).head(12))

Top topicCategories kept: lifestyle_(sociology)      966468
physical_attractiveness    963952
fashion                     39234
hobby                       22751
health                      15703
entertainment               11805
pop_music                   10784
music_of_asia                5828
physical_fitness             2384
society                      2294
sport                         993
television_program            810
dtype: int64


In [38]:
# Aggregate rescue analysis (all English rows)
n_other_kw_total = (all_df_en["category_kw"] == "other").sum()
rescued_total = ((all_df_en["category_kw"] == "other") & 
                 (all_df_en["category_ml"] != "other")).sum()

print("Total OTHER by keyword:", n_other_kw_total)
print("Total rescued by ML:", rescued_total)
print("Rescue rate:", round(rescued_total / max(n_other_kw_total, 1) * 100, 2), "%")

print("\nFinal rescued distribution:")
print(all_df_en.loc[all_df_en["category_kw"]=="other", "category_ml"].value_counts().head(10))

print("\nFinal category distribution (category_final):")
print(all_df_en["category_final"].value_counts())


Total OTHER by keyword: 103990
Total rescued by ML: 103990
Rescue rate: 100.0 %

Final rescued distribution:
category_ml
makeup            67474
hair              26039
skincare           5370
beauty_general     5095
fragrance            12
Name: count, dtype: int64

Final category distribution (category_final):
category_final
makeup            401230
skincare          270236
hair              201599
beauty_general     29867
fragrance           7805
Name: count, dtype: int64


In [39]:
# Final combined dataset + aggregated metrics

# Ensure category_final exists and is consistent
assert "category_final" in all_df_en.columns, "category_final missing in all_df_en"

# Combine All English
all_df_final = pd.concat([all_df_en], ignore_index=True)

print("Final combined shape:", all_df_final.shape)
print("\nCategory distribution (final):")
print(all_df_final["category_final"].value_counts())

Final combined shape: (910737, 20)

Category distribution (final):
category_final
makeup            401230
skincare          270236
hair              201599
beauty_general     29867
fragrance           7805
Name: count, dtype: int64


In [40]:
all_df_final.head()

videoId                                       textOriginal  \
0    74288  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would ro...   
1    79618   Apply mashed potato juice and mixed it with curd   
2    51826                         69 missed calls from mars👽   
3    58298                                               Baaa   
4     1265    you look like raven from phenomena raven no cap   

                                               title  \
0  I tried hair inspired by the PAN flag 🩷💛🩵 #pan...   
1  5 Foundation Mistakes that Every Girl Should Know   
2                 How To Make Small Eyes Look Bigger   
3  20sec beauty test: BLUSH PLACEMENT for YOUR FA...   
4               BLACK GIRL TRIES KYLIE JENNER MAKEUP   

                                         description  \
0                                                NaN   
1  5 Foundation Mistakes that Every Girl Should K...   
2                 How To Make Small Eyes Look Bigger   
3                                                NaN   
4  Today on Black Girl Tries we are trying Kylie ...   

                                                tags  \
0                                                NaN   
1  ['makeup mistakes to avoid', 'common makeup mi...   
2  ['How To Make Small Eyes Look Bigger', 'eye ma...   
3                                                NaN   
4  ['kylie jenner', 'kylie cosmetics', 'black gir...   

                                     topicCategories  \
0  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
1  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
2  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
3  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   
4  ['https://en.wikipedia.org/wiki/Lifestyle_(soc...   

                                 textOriginal_clean  \
0   please lesbian flag i beg you you would rock it   
1  apply mashed potato juice and mixed it with curd   
2                         69 missed calls from mars   
3                                              baaa   
4   you look like raven from phenomena raven no cap   

                                         title_clean  \
0  i tried hair inspired by the pan flag pansexua...   
1  5 foundation mistakes that every girl should know   
2                 how to make small eyes look bigger   
3  20sec beauty test blush placement for your fac...   
4               black girl tries kylie jenner makeup   

                                   description_clean  \
0                                                      
1  5 foundation mistakes that every girl should k...   
2                 how to make small eyes look bigger   
3                                                      
4  today on black girl tries we are trying kylie ...   

                                          tags_clean  \
0                                                      
1  makeup mistakes to avoid common makeup mistake...   
2  how to make small eyes look bigger eye makeup ...   
3                                                      
4  kylie jenner kylie cosmetics black girl tries ...   

                              topicCategories_clean  \
0  [Lifestyle_(sociology), Physical_attractiveness]   
1  [Lifestyle_(sociology), Physical_attractiveness]   
2  [Lifestyle_(sociology), Physical_attractiveness]   
3  [Lifestyle_(sociology), Physical_attractiveness]   
4  [Lifestyle_(sociology), Physical_attractiveness]   

                           topicCategories_clean_lc  \
0  [lifestyle_(sociology), physical_attractiveness]   
1  [lifestyle_(sociology), physical_attractiveness]   
2  [lifestyle_(sociology), physical_attractiveness]   
3  [lifestyle_(sociology), physical_attractiveness]   
4  [lifestyle_(sociology), physical_attractiveness]   

   is_english__textOriginal_clean  is_english__title_clean  \
0                            True                     True   
1                            True                     True   
2                           False                     True   
3           

## ⚙️ Environment Setup

Before running the multilingual translation demo, install the required dependencies:

```bash
pip install -q transformers torch accelerate pandas langid ftfy emoji scikit-learn


In [41]:
pip install -q transformers torch accelerate pandas langid ftfy emoji scikit-learn


Note: you may need to restart the kernel to use updated packages.


## 🌍 Multilingual Translation Demo

Although our main pipeline focused on English comments, we demonstrated the ability to handle multiple languages.  
Using the **M2M100 multilingual model**, we automatically detected and translated **1,000 non-English comments** into English.  

This ensures that in the future, **every market’s consumer voice** can be analyzed consistently under the same framework.

### Process
1. Detect language of each comment (`langid`).
2. Map to supported M2M100 codes (`lang_m2m`).
3. Translate non-English comments → English (`en_text`).
4. Preserve original + translation for audit.

### Sample of Translated Comments

| Original (textOriginal)       | Detected | Mapped | Translated (en_text)               |
|-------------------------------|----------|--------|-------------------------------------|
| भाई खुश हो गया 😂              | hi       | hi     | Brother is happy 😂                 |
| Cantik banget yak Allah ❤❤     | id       | id     | So beautiful, oh God ❤❤             |
| 여깄음                          | ko       | ko     | Here it is                          |
| T trop belle je voudrais ...  | fr       | fr     | You're so beautiful I would love... |
| JazakaAllah stay blessed love | ms       | ms     | JazakaAllah stay blessed love you   |

➡️ **Outcome:** A unified English text stream, ready for sentiment, topic categorization, and KPI analysis across languages.  


In [77]:
# === 1K MULTILINGUAL TRANSLATION DEMO (robust) ===
# Requires: transformers, ftfy, emoji, langid (or your existing lang column)

import re, numpy as np, pandas as pd, torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 0) Pick DF and text column
try:
    df = df_comments.copy()
except NameError:
    raise RuntimeError("Define df_comments first.")

TEXT_CANDIDATES = ["textOriginal", "comment_text", "text", "body", "content", "message"]
COMMENT_COL = next((c for c in TEXT_CANDIDATES if c in df.columns), None)
if not COMMENT_COL:
    raise RuntimeError(f"None of {TEXT_CANDIDATES} found in columns: {list(df.columns)[:30]}")

# 1) Clean + language detection (fallback to langid if you don't have 'lang')
import ftfy, emoji
def clean_text(t):
    if not isinstance(t, str): return ""
    t = ftfy.fix_text(t).replace("\u200b"," ").strip()
    t = emoji.replace_emoji(t, replace=" ")
    t = re.sub(r"\s+", " ", t)
    return t

df["text_clean"] = df[COMMENT_COL].map(clean_text)
if "lang" not in df.columns:
    import langid
    langid.set_languages(None)
    df["lang"] = df["text_clean"].map(lambda s: langid.classify(s[:500])[0] if s else "und")

# 2) Canonicalize to M2M100 language codes
tok_mt = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
supported = set(tok_mt.lang_code_to_id.keys())
ALIASES = {"zh-cn":"zh","zh-hans":"zh","zh-hant":"zh","jp":"ja","fil":"tl"}

DEVANAGARI = re.compile(r"[\u0900-\u097F]")  # hi/ne/…
ARABIC     = re.compile(r"[\u0600-\u06FF]")
CYRILLIC   = re.compile(r"[\u0400-\u04FF]")

def canonical_lang(code, text):
    code = (code or "").lower()
    code = ALIASES.get(code, code)
    if code in supported: return code
    t = text or ""
    if DEVANAGARI.search(t) and "hi" in supported: return "hi"
    if ARABIC.search(t) and "ar" in supported:     return "ar"
    if CYRILLIC.search(t) and "ru" in supported:   return "ru"
    return "en"

df["lang_m2m"] = [canonical_lang(c, t) for c, t in zip(df["lang"], df["text_clean"])]

# 3) Take exactly 1,000 non-English rows for the demo
demo = df[(df["text_clean"].str.len() >= 3) & (df["lang_m2m"] != "en")].sample(
    n=min(1000, ((df["lang_m2m"] != "en") & df["text_clean"].str.len().ge(3)).sum()),
    random_state=42
).reset_index(drop=True)

# 4) Translate with robust generate() path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mod_mt = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M").to(device)
forced_en = tok_mt.lang_code_to_id["en"]
MAX_LEN, BATCH_TR = 128, 64

def translate_group(sub):
    src = str(sub["lang_m2m"].iat[0]) or "en"
    tok_mt.src_lang = src
    texts = sub["text_clean"].tolist()
    outs = []
    for i in range(0, len(texts), BATCH_TR):
        batch = texts[i:i+BATCH_TR]
        enc = tok_mt(batch, return_tensors="pt", padding=True,
                     truncation=True, max_length=MAX_LEN).to(device)
        gen = mod_mt.generate(**enc, forced_bos_token_id=forced_en, max_length=MAX_LEN)
        dec = tok_mt.batch_decode(gen, skip_special_tokens=True)
        outs.extend(dec)
    return sub.assign(en_text=outs)

parts = []
for lang_code, grp in demo.groupby("lang_m2m", sort=False):
    try:
        parts.append(translate_group(grp))
    except Exception:
        # In case of rare failures, keep original to avoid blocking the demo
        parts.append(grp.assign(en_text=grp["text_clean"]))
demo_tr = pd.concat(parts, ignore_index=True)

# 5) Save + show a clean sample
out_csv = "translate_only_demo_1k.csv"
demo_tr[[COMMENT_COL, "lang", "lang_m2m", "text_clean", "en_text"]].to_csv(out_csv, index=False)

# Only rows where translation actually changed
changed = demo_tr[demo_tr["en_text"].ne(demo_tr["text_clean"])]

print(f"✅ Saved: {out_csv}")
print(f"Rows translated (non-EN in demo): {len(demo_tr)}")
print(f"Actually changed by MT: {len(changed)}")
print("\nSample translated rows:")
display(changed[[COMMENT_COL, "lang", "lang_m2m", "en_text"]].head(15))

print("\nTop languages in 1k demo:")
display(demo_tr["lang_m2m"].value_counts().head(10))


✅ Saved: translate_only_demo_1k.csv
Rows translated (non-EN in demo): 1000
Actually changed by MT: 921

Sample translated rows:


textOriginal lang lang_m2m  \
0              Que interesante es la belleza genuina.   es       es   
1                He look like a real Sugar Daddy now.   es       es   
2                           India ka makeup acha laga   es       es   
3                                       Tu la u ou???   es       es   
4   Wow que increible shampu amiga realmente cubre...   es       es   
5   Necesito que me maquilles ya que soy pésima ma...   es       es   
6   Admiro tu trabajo no se maquillarme pero me en...   es       es   
7                       No hay subtitulado al español   es       es   
8   Es muy linda con o sin cabello pero se podría ...   es       es   
9                                        Ermosa❤❤❤❤🎉😊   es       es   
10                                        I indonesia   es       es   
12                                      Bangla desh❤️   es       es   
13  Pense que iba a transformarse en blanca hasta ...   es       es   
14                    I just love ur vdos❤️❤️❤️❤️👌👌👌👌   es       es   
15                                         Im aurora❤   es       es   

                                              en_text  
0                 How interesting is the real beauty.  
1               He looks like a real Sugar Daddy now.  
2                             India to make acha laga  
3                                       You or you???  
4   Wow that incredible shampoo friend really cove...  
5               I need makeup because I’m bad makeup.  
6   I admire your work don't make me but I love to...  
7                             No subtitles in Spanish  
8   It’s very beautiful with or without hair but y...  
9                                           by Ermosa  
10                                      and Indonesia  
12                                    The Bangla desh  
13  I thought I was going to be white until I read...  
14                                    I just love you  
15                                          by aurora


Top languages in 1k demo:


lang_m2m
es    123
it     72
de     70
id     68
sw     60
fr     57
ar     46
pt     44
ru     35
fi     32
Name: count, dtype: int64

## ✅ Relevance Scoring (Simple Proof)

To demonstrate that translated comments are actually relevant to L’Oréal’s categories,  
we computed a **simple keyword-based Relevance Score**:

- Score = number of category keywords found / 5 (capped at 1.0).  
- Flag `is_relevant = True` if score > 0.  
- Keywords cover skincare, makeup, fragrance, and haircare.

**Outcome:**  
Even this simple method shows that a portion of translated comments are directly tied to L’Oréal’s core product areas — validating that a full multilingual rollout is worthwhile.


In [97]:
import re

# Use translated text
TEXT_COL = "en_text"

# Small keyword list for L'Oréal categories
KEYWORDS = [
    "makeup","skincare","serum","foundation","lipstick",
    "mascara","eyeliner","perfume","fragrance","shampoo",
    "conditioner","hair","cream","toner"
]

def relevance_score(text, keywords=KEYWORDS):
    if not isinstance(text, str):
        return 0
    text = text.lower()
    hits = sum(1 for kw in keywords if kw in text)
    return min(hits, 5) / 5  # normalize 0–1

# Apply only on the translated demo (demo_tr)
demo_tr["relevance_score"] = demo_tr[TEXT_COL].map(relevance_score)
demo_tr["is_relevant"] = demo_tr["relevance_score"] > 0

# Quick summary
print("Overall relevance ratio (translated only):", demo_tr["is_relevant"].mean())
print(demo_tr["relevance_score"].describe())

# See first 20 rows
print(demo_tr[[TEXT_COL,"relevance_score","is_relevant"]].head(20))


Overall relevance ratio (translated only): 0.101
count    1000.000000
mean        0.020800
std         0.063018
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         0.400000
Name: relevance_score, dtype: float64
                                              en_text  relevance_score  \
0                 How interesting is the real beauty.              0.0   
1               He looks like a real Sugar Daddy now.              0.0   
2                             India to make acha laga              0.0   
3                                       You or you???              0.0   
4   Wow that incredible shampoo friend really cove...              0.2   
5               I need makeup because I’m bad makeup.              0.2   
6   I admire your work don't make me but I love to...              0.2   
7                             No subtitles in Spanish              0.0   
8   It’s very beautiful with or without hair but y...              0.2   
9    

## 🌍 Multilingual Translation Demo (Future Implementation)

As part of future planning, we ran a **proof-of-concept translation** of **1,000 non-English comments** into English using a multilingual AI model.  

### What we did
- ✅ Translated 1,000 rows as a demo.  
- ✅ Computed a **Relevance Score** → only **11.8% (0.118365)** of translated comments were actually relevant to L’Oréal’s core categories.  

### What this means
- ❓ At this stage, it’s **unclear if scaling full translation is cost-effective**, since most comments remain irrelevant noise.  
- 🔎 However, this is valuable for **trend monitoring**: if relevance trends increase in non-English data, it may justify expanding translation efforts.  
- ☁️ In that case, we recommend integrating **cloud translation services (e.g., Google Cloud Translate, AWS Translate)** to handle scale efficiently across all markets.  

### Business Insight
- Current engagement is heavily noisy → full-scale translation today might not deliver strong ROI.  
- But maintaining multilingual readiness ensures **L’Oréal can capture emerging insights in global markets** without missing signals.  
- **Decision point:** treat this as an **option for scaling** — we can flip on cloud translation if and when the data shows higher relevance.  

➡️ **Takeaway:** We proved translation is technically feasible. The next step is a **business-driven decision**:  
scale only if non-English engagement shows meaningful trends that justify the cost.  
